In [ ]:
import os  # used for directory operations
import tensorflow as tf
from PIL import Image  # used to read images from directory
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:

# path to record files
cwd = Path('..')
# path to folder to save tfrecords
recordPath = Path('..')
count =  0
# number of pictures included in single record
# There should be 10 times as many files as there will be hosts processing the data
# The files should be 100 MB + size to fully use prefetch benefits
bestNum = 1000
# the index of images flowing into each tfrecord file
num = 0
# the index of the tfrecord file
recordFileNum = 0
# dictionary with labels encoded as numbers
keys = []
values = []
for folder in os.listdir(cwd):
    keys.append(folder)
keys = sorted(keys)
for i in range(0,len(keys)):
    values.append(i)
classes = dict(zip(keys, values))
# # name format of the tfrecord files
# recordFileName = 'dataset.tfrecords'
writer = tf.io.TFRecordWriter(recordPath + str(recordFileNum))

In [ ]:
print(classes)
print(keys)

In [ ]:
## functions creating features of certain types required to write a tfrecord
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [ ]:
def serialize_example(image, label, image_shape):
    feature = {
        'image': _bytes_feature(image),
        'label': _int64_feature(label),
        'height': _int64_feature(image_shape[0]),
        'width': _int64_feature(image_shape[1]),
        'depth': _int64_feature(image_shape[2]),
    }
#  Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
for name, label in classes.items():
    class_path = os.path.join(cwd, name)
    for img_name in os.listdir(class_path):
        num += 1
        if num > bestNum:
            num = 1
            recordFileNum += 1
            writer = tf.io.TFRecordWriter(recordPath + str(recordFileNum)+str(name))
        print("Creating the %.3d tfrecord file" % recordFileNum)
        img_path = os.path.join(class_path, img_name)
        img = tf.keras.preprocessing.image.load_img(img_path)
        img = tf.image.resize(img,[100,100]) ## load image
        img_array = tf.keras.preprocessing.image.img_to_array(img) ## convert to array
        img_array = img_array.astype(np.float32) ## convert values to float
        img_raw = tf.io.serialize_tensor(img_array) ## serialize tensor into string of bytes
        img_shape = img_array.shape ## recover shape of image
        example = serialize_example(img_raw, label, img_shape) ## use defined function
        ## to serailize example and prepare it for writing to tfrecord
        writer.write(example)
writer.close()
print("Creating dataset finished.")